In [2]:
"""
You are given an n x n binary matrix grid where 1 represents 
land and 0 represents water. An island is a 4-directionally connected 
group of 1's not connected to any other 1's. There are exactly two 
islands in grid. You may change 0's to 1's to connect the two islands 
to form one island. Return the smallest number of 0's you must flip 
to connect the two islands.


Example 1:
    Input: 
        grid = [[0,1],[1,0]]
    Output: 
        1

Example 2:
    Input: 
        grid = [[0,1,0],[0,0,0],[0,0,1]]
    Output: 
        2

Example 3:
    Input: 
        grid = [[1,1,1,1,1],[1,0,0,0,1],[1,0,1,0,1],[1,0,0,0,1],[1,1,1,1,1]]
    Output: 
        1
 

Constraints:
    n == grid.length == grid[i].length
    2 <= n <= 100
    grid[i][j] is either 0 or 1.
    There are exactly two islands in grid.
"""

# Works fine now because not visiting anything again.

from collections import deque
from typing      import List

class Solution:
    def shortestBridge(self, grid: List[List[int]]) -> int:
        R = len(grid)
        C = len(grid[0])

        def get_island(i, j, visited):
            if (
                i < 0 or i >= R or 
                j < 0 or j >= C or 
                (i, j) in visited or
                grid[i][j] == 0
            ):
                return
            island1.add((i, j))
            visited.add((i, j))
            get_island(i+1, j, visited)
            get_island(i-1, j, visited)
            get_island(i, j+1, visited)
            get_island(i, j-1, visited)

        x, y = -1, -1
        for loc in range(R*C):
            r, c = int(loc/C), int(loc%C)
            if grid[r][c] == 1:
                x, y = r, c
                break

        island1 = set()
        get_island(x, y, set())
        islandi = deque([((i, j), 0) for i, j in island1])
        visited = set(island1)
        not_yet = True
        bridgel = None
        while islandi and not_yet:
            (ci, cj), diff = islandi.pop()
            for dx, dy in [(1, 0), (-1, 0), (0, -1), (0, 1)]:
                p, q = ci+dx, cj+dy
                if p < 0 or p >= R or q < 0 or q >= C or (p, q) in visited:
                    continue
                val = grid[p][q]
                if val == 1:
                    not_yet = False
                    bridgel = diff
                    break
                visited.add((p, q))
                islandi.appendleft(((p, q), diff+1))
        return bridgel


In [ ]:

from itertools import product
# Fastest --- 2 BFS
class Solution:
    def shortestBridge(self, grid: List[List[int]]) -> int:
        n, ans = len(grid), 1
        def neighbors(cell):
            i, j = cell
            if i > 0 and grid[i-1][j] != -1: yield i-1, j
            if i < n-1 and grid[i+1][j] != -1: yield i+1, j
            if j > 0 and grid[i][j-1] != -1: yield i, j-1
            if j < n-1 and grid[i][j+1] != -1: yield i, j+1
        def island():
            for i, j in product(range(n), range(n)):
                if grid[i][j]:
                    grid[i][j] = -1
                    return [(i, j)]
        q, q2 = island(), []
        while q:
            q_next = []
            for cell in q:
                for i, j in neighbors(cell):
                    (q_next if grid[i][j] else q2).append((i, j))
                    grid[i][j] = -1
            q = q_next
        while q2:
            q_next = []
            for cell in q2:
                for i, j in neighbors(cell):
                    if grid[i][j] == 1: return ans
                    q_next.append((i, j))
                    grid[i][j] = -1
            ans += 1
            q2 = q_next
        return ans

In [ ]:


# TLE -- because applying backtrack trick of removing visited on a path.
from collections import deque
from typing      import List

class Solution:
    def shortestBridge(self, grid: List[List[int]]) -> int:
        R = len(grid)
        C = len(grid[0])

        def get_nbrs(i, j, visited):
            if (
                i < 0 or i >= R or 
                j < 0 or j >= C or 
                (i, j) in visited or
                grid[i][j] == 0
            ):
                return
            island1.add((i, j))
            visited.add((i, j))
            get_nbrs(i+1, j, visited)
            get_nbrs(i-1, j, visited)
            get_nbrs(i, j+1, visited)
            get_nbrs(i, j-1, visited)
            visited.remove((i, j))

        def my_nbrs(i, j, visited):
            nbrs = []
            for dx, dy in [(1, 0), (-1, 0), (0, -1), (0, 1)]:
                p, q = i+dx, j+dy
                if p < 0 or p >= R or q < 0 or q >= C or (p, q) in visited:
                    continue
                nbrs.append((p, q, grid[p][q]))
            return nbrs

        x, y = -1, -1
        for loc in range(R*C):
            r, c = int(loc/C), int(loc%C)
            if grid[r][c] == 1:
                x, y = r, c
                break

        island1 = set()
        get_nbrs(x, y, set())
        islandi = deque([((i, j), 0) for i, j in island1])
        visited = set(island1)
        not_yet = True
        bridgel = None
        while islandi and not_yet:
            (ci, cj), diff = islandi.pop()
            nbrs = my_nbrs(ci, cj, visited)
            for ni, nj, val in nbrs:
                if val == 1:
                    not_yet = False
                    bridgel = diff
                    break
                visited.add((ni, nj))
                islandi.appendleft(((ni, nj), diff+1))
        return bridgel